In [ ]:
!nvidia-smi

Tue Mar 11 12:34:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install PyMuPDF
!pip install langchain
!pip install faiss-cpu
!pip install transformers
!pip install numpy
!pip install tqdm
!pip install rank-bm25

In [ ]:
import fitz
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModel
import numpy as np
import faiss
import torch
from tqdm import tqdm
import pickle
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    pdf = []
    for page, text in enumerate(doc):
        # text += page.get_text("text") + "\n"  # Extract text from each page
        pdf.append({
            'text' : text.get_text('text'),
            'page' : page
        })
    return pdf

# pdf_text = extract_text_from_pdf(r"D:\legal_dataset\Ajay_Kumar_Singla_Ors_vs_State_Of_Punjab_Ors_on_16_February_2023.PDF")

In [ ]:
# pdf_text

[{'text': 'Ajay Kumar Singla & Ors vs State Of Punjab & Ors on 16\nFebruary, 2023\nAuthor: M.S. Ramachandra Rao\nBench: M.S. Ramachandra Rao\n                                                Neutral Citation No:=2023:PHHC:029128-DB\n       IN THE HIGH COURT OF PUNJAB AND HARYANA AT\n                           CHANDIGARH\n                              ****\n1.                                       CWP-17064-2017\nAjay Kumar Singla & Others\n                                            .... Petitioners\n                              Versus\nState of Punjab and Others\n                                           .... Respondents\n                              ****\n2.                                       CWP-6232-2017\nMohammad Daud Alam and Others\n                                            .... Petitioners\n                              Versus\nState of Punjab and Others\n                                           .... Respondents\n                              ****\n3.                  

In [ ]:
legal_data = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
for path, folders, files in os.walk("/content/drive/MyDrive/legal_dataset"):
    for file in files:
        extracted_pdf = extract_text_from_pdf(os.path.join(path,file))
        for page in extracted_pdf:
            chunks = text_splitter.split_text(page['text'])
            for chunk in chunks:
                legal_data.append({
                    'chunk' : chunk,
                    'page' : page['page'],
                    'source' : file
                })

In [ ]:
legal_data[0], len(legal_data)

({'chunk': 'THE COMMERCIAL COURTS ACT, 2015\nUNION OF INDIA\nIndia\nTHE COMMERCIAL COURTS ACT, 2015\nAct 04 of 2016\nPublished in Gazette of India on 31 December 2015\n• \nNot commenced\n• \n[This is the version of this document from 31 December 2015.]\n• \n[Note: The original publication document is not available and this content could not be\nverified.]\n• \nTHE COMMERCIAL COURTS, COMMERCIAL DIVISION AND COMMERCIAL APPELLATE\nDIVISION OF HIGH COURTS ACT , 2015NO. 4 OF 2016[31st December,2015.]An Act to provide',
  'page': 0,
  'source': 'THE_COMMERCIAL_COURTS_ACT_2015.PDF'},
 68895)

In [ ]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en")
model = AutoModel.from_pretrained("BAAI/bge-large-en")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

In [ ]:
for chunked_data in tqdm(legal_data, total=len(legal_data), desc="Embedding Chunks", unit="chunk"):
    # Tokenize
    encoded_input = tokenizer(chunked_data['chunk'], return_tensors="pt", truncation=True, padding=True)
    encoded_input = {key: value.to(device) for key, value in encoded_input.items()}

    # Get embeddings
    with torch.no_grad():
        output = model(**encoded_input)

    # Mean pooling
    sentence_embedding = torch.mean(output.last_hidden_state, dim=1).squeeze(0).cpu().numpy()

    # Store chunk with metadata
    # chunk_data.append({
    #     "chunk": chunk,                      # Text of the chunk
    #     "embedding": sentence_embedding,      # Vector representation
    #     "document": "example.pdf",            # Replace with actual PDF name
    #     "page": 5,                            # Replace with actual page number
    #     "chunk_index": i                       # Position of chunk within the page
    # })
    chunked_data['embedding'] = sentence_embedding


Embedding Chunks: 100%|██████████| 68895/68895 [32:34<00:00, 35.26chunk/s]


In [ ]:
dimension = legal_data[0]['embedding'].shape[0]
index = faiss.IndexFlatL2(dimension)
embedding_vectors = np.array([item["embedding"] for item in tqdm(legal_data, total=len(legal_data), desc="Embedding Chunks", unit="chunk")]).astype('float32')

index.add(embedding_vectors)

faiss.write_index(index, "faiss_index.bin")

with open("metadata.pkl", "wb") as f:
    pickle.dump(legal_data, f)

Embedding Chunks: 100%|██████████| 68895/68895 [00:00<00:00, 1840563.15chunk/s]


In [ ]:
index = faiss.read_index(r"C:\Users\shobh\Downloads\faiss_index.bin")

with open(r"C:\Users\shobh\Downloads\metadata.pkl", "rb") as f:
    legal_data = pickle.load(f)


tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en")
model = AutoModel.from_pretrained("BAAI/bge-large-en").to("cuda" if torch.cuda.is_available() else "cpu")

def embed_query(query):
    encoded_input = tokenizer(query, return_tensors="pt", truncation=True, padding=True).to("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        output = model(**encoded_input)
    return torch.mean(output.last_hidden_state, dim=1).squeeze(0).cpu().numpy()

query = "Legality of income tax reassessment under Section 153C"

query_vector = embed_query(query).reshape(1, -1).astype('float32')

k = 5  # Number of results
distances, indices = index.search(query_vector, k)

top_results = [legal_data[i] for i in indices[0]]


for i, result in enumerate(top_results):
    print(f"Rank {i+1}\nSource: {result['source']} \nPage {result['page']} - {result['chunk'][:200]}...\n")

Rank 1
Source: Honda_Siel_Power_Products_Limited_vs_The_Deputy_Commissioner_Of_Income_Tax_on_14_February_2011.PDF 
Page 5 - section 148 of the Act initiating the reassessment proceedings, therefore, could
validly be issued till 31-03-2005 in terms of the proviso to section 147 of the Act. In
Honda Siel Power Products Limit...

Rank 2
Source: Pushpa_Rathi_vs_Income_Tax_Officer_Ward_30_1_Anr_on_10_November_2023.PDF 
Page 35 - under:-
                              xxx                                xxx                    
                              (iii)      Section 147 proposes to allow the Assessing Off
reassess or re...

Rank 3
Source: Parnika_Rathi_vs_Income_Tax_Officer_Ward_30_1_Delhi_on_10_November_2023.PDF 
Page 35 - under:-
                              xxx                                xxx                    
                              (iii)      Section 147 proposes to allow the Assessing Off
reassess or re...

Rank 4
Source: Devendra_Kumar_Saini_vs_Income_Tax_Officer_Wa

In [ ]:
legal_data[:10]

[{'chunk': 'THE COMMERCIAL COURTS ACT, 2015\nUNION OF INDIA\nIndia\nTHE COMMERCIAL COURTS ACT, 2015\nAct 04 of 2016\nPublished in Gazette of India on 31 December 2015\n• \nNot commenced\n• \n[This is the version of this document from 31 December 2015.]\n• \n[Note: The original publication document is not available and this content could not be\nverified.]\n• \nTHE COMMERCIAL COURTS, COMMERCIAL DIVISION AND COMMERCIAL APPELLATE\nDIVISION OF HIGH COURTS ACT , 2015NO. 4 OF 2016[31st December,2015.]An Act to provide',
  'page': 0,
  'source': 'THE_COMMERCIAL_COURTS_ACT_2015.PDF',
  'embedding': array([-0.7368621 , -0.5636795 , -0.33564132, ...,  0.47133026,
         -0.20095436, -0.67343414], dtype=float32)},
 {'chunk': 'DIVISION OF HIGH COURTS ACT , 2015NO. 4 OF 2016[31st December,2015.]An Act to provide\nfor the constitution of Commercial Courts, Commercial Division and Commercial Appellate Division\nin the High Courts for adjudicating commercial disputes of specified value and matters c

In [ ]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\shobh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True